In [2]:
!pip install crewai crewai-tools
!pip install langchain
!pip install qdrant-client
!pip install langchain-groq
!pip install sentence-transformers

  Obtaining dependency information for crewai from https://files.pythonhosted.org/packages/54/f2/1e67c9392779cca081566b0845ab12be915d0d420f0133313ed92d5e97b2/crewai-0.41.1-py3-none-any.whl.metadata
  Obtaining dependency information for crewai-tools from https://files.pythonhosted.org/packages/82/68/5e0262b7010292fd5df3f707a037c5e4a2cae65b61333e995cfe09adedbb/crewai_tools-0.4.26-py3-none-any.whl.metadata
  Obtaining dependency information for appdirs<2.0.0,>=1.4.4 from https://files.pythonhosted.org/packages/3b/00/2344469e2084fb287c2e0b57b72910309874c3245463acd6cf5e3db69324/appdirs-1.4.4-py2.py3-none-any.whl.metadata
  Obtaining dependency information for click<9.0.0,>=8.1.7 from https://files.pythonhosted.org/packages/00/2e/d53fa4befbf2cfa713304affc7ca780ce4fc1fd8710527771b58311a3229/click-8.1.7-py3-none-any.whl.metadata
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Obtaining dependency information for embedchain<0.2.0,>=0.1.114 from https://files.pythonhosted.org/pa

## Load data

In [3]:
import pandas as pd
df = pd.read_csv("adidas.csv", sep=';')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2625 entries, 0 to 2624
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   URL            2625 non-null   object
 1   Product Name   2625 non-null   object
 2   Product ID     2625 non-null   object
 3   Listing Price  2625 non-null   int64 
 4   Sale Price     2625 non-null   int64 
 5   Discount       2625 non-null   int64 
 6   Brand          2625 non-null   object
 7   Description    2625 non-null   object
 8   Rating         2625 non-null   int64 
 9   Reviews        2625 non-null   int64 
 10  Images         2625 non-null   object
 11  Last Visited   2625 non-null   object
dtypes: int64(5), object(7)
memory usage: 246.2+ KB


In [5]:
df.Brand.value_counts()

Brand
CORE / NEO           1111
ORIGINALS             908
SPORT PERFORMANCE     606
Name: count, dtype: int64

In [6]:
def extended_descriptin(row):
    ex_desc = f"Poduct Name: {row['Product Name'].strip()} \nDescription of product: {row['Description'].strip()}"
    return ex_desc

In [7]:
df['ExtendedDescription'] = df.apply(extended_descriptin, axis=1)

In [8]:
texts = df['ExtendedDescription'].to_list()

In [9]:
print(texts[0],"\n")

Poduct Name: Women's adidas Originals NMD_Racer Primeknit Shoes 
Description of product: Channeling the streamlined look of an '80s racer, these shoes are updated with modern features. The foot-hugging adidas Primeknit upper offers a soft, breathable feel. The Boost midsole provides responsive comfort accented with a contrast-color EVA heel plug. Embroidered details add a distinctive finish. 



## Encode Texts

In [10]:
import torch
from sentence_transformers import SentenceTransformer
device_to_use = torch.device('cuda') if torch.cuda.is_available() else "cpu"
model = SentenceTransformer("BAAI/bge-large-en-v1.5", device=device_to_use)
vectors = model.encode(texts)

/home/tsnanh/PycharmProjects/landing-page-generator/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


## Qdrant Setup

In [19]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

# Initialize the client

client = QdrantClient(":memory:",)
client.create_collection(
    collection_name="adidas_products",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
)

client.upload_collection(
    collection_name="adidas_products",
    ids=[i for i in range(len(texts))],
    vectors=vectors,
    parallel=4,
    max_retries=3,
)

In [20]:
def get_matches(text):
    query_vector = model.encode(text)
    hits = client.search(
        collection_name="adidas_products",
        query_vector=query_vector,
        limit=20 # Return 20 closest points
    )
    products = list(set([df['Product Name'][i.id] for i in hits]))
    descriptions = list(set([df['Description'][i.id] for i in hits]))
    desc=""
    pdts=""
    for x, y in zip(descriptions, products):
        desc = desc + x + "\n"
        pdts = pdts + y + ', '
    return desc, pdts

In [21]:
print(get_matches("men tennis shoes"))

("These tennis-inspired shoes elevate their style game off the court with a knit upper and decorative laces. The slip-on look has a seamless, sock-like construction with an ultra-soft Cloudfoam sockliner for added comfort throughout your day.\nServe up some style when you hit the club. The full synthetic upper provides unmatched durability. A cushioned EVA midsole keeps you comfortable from the moment you pull them on. The durable outsole stands up to every serve and slide as you improve your game.\nYou play every single day so you need a shoe that's both durable and comfortable. These adidas tennis shoes have a cushioned midsole for enhanced comfort whether you're working on your groundstroke or playing best-of-five. A durable and supportive build lets you play on any type of court surface.\nThe adidas steadfast tennis shoes for men. A new, modern design that delivers quality and comfort for the recreational tennis player. The Synthetic & Mesh combination on the upper makes it lightwe

## Groq API

In [22]:
from langchain_groq import ChatGroq

In [28]:
groq = ChatGroq(temperature=0.1, groq_api_key="API_KEY", 
                model_name="llama3-8b-8192")

In [29]:
# system = "You are a helpful assistant."
# human = "{text}"
# prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

# chain = prompt | groq
# chain.invoke({"text": "Explain the importance of low latency LLMs."})

In [30]:
from langchain_community.chat_models import ChatOllama

local_llm = "phi3"

ollama = ChatOllama(model=local_llm, temperature=0)

## CrewAI system integration

In [31]:
from crewai import Agent, Task, Crew

def agent_pipeline(query):
    desc, products = get_matches(query)
    tagline_agent = Agent(role = "Marketing and Copy writing Head",
                        goal = """Provide a catchy and creative tagline from the description that is provided JUST A TAGLINE.""",
                        backstory = """You are an excellent Marketing and Copy writing genius that doesnt explain things just gives results, who can sell any product via marketing the shit out of it.""",
                        allow_delegation = False,
                        verbose = True,
                        llm = groq)

    explainindetail_agent = Agent(role = "Senior Marketing and Copy writing Executive",
                        goal = """Provide the pros of using this product according to the product and highlight the strong points..""",
                        backstory = """You are an excellent Marketing and Copy writing genius that doesnt explain things just gives results, who can sell any product via marketing the shit out of it.""",
                        allow_delegation = False,
                        verbose = True,
                        llm = groq)

    listproducts_agent = Agent(role = "Marketing and Copy writing Analyst",
                        goal = """You'll be given a list of products, list them in presentable manner with 1 line each that's it.""",
                        backstory = """You are an excellent Marketing and Copy writing genius that doesnt explain things just gives results, who can sell any product via marketing the shit out of it.""",
                        allow_delegation = False,
                        verbose = True,
                        llm = groq)
    # manager_agent = Agent(role = "Manager",
    #                     goal = """You'll be managing all the other agents.""",
    #                     backstory = """You are an excellent man power management genius that doesnt explain things just gives results, who can maager any team via managing the shit out of it.""",
    #                     allow_delegation = False,
    #                     verbose = True,
    #                     llm = groq,
    #                     )


    task1 = Task (description=desc,
                agent = tagline_agent,
                expected_output="A tagline for the product.")

    task2 = Task (description=desc,
                agent = explainindetail_agent,
                expected_output="A description for the product.")

    task3 = Task(description=products,
                agent=listproducts_agent,
                expected_output="A numbered 1,2,3,.. list of products from the provided list.")

    results = []
    for agent, task in zip([tagline_agent, explainindetail_agent, listproducts_agent],[task1, task2, task3]):
        crew1 = Crew(agents=[agent],
                tasks=[task],
                verbose=3,
                # manager_llm=groq
            )
        result = crew1.kickoff()
        results.append(result)
    
    return {'tagline':results[0],'product_description':results[1],'products_list':results[2]}    

## Inference

In [32]:
result = agent_pipeline(query="men's casual shoes")

2024-08-07 00:43:29,319 - 138031927712640 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-08-07 00:43:29][DEBUG]: == Working Agent: Marketing and Copy writing Head
 [2024-08-07 00:43:29][INFO]: == Starting Task: The adidas casual sandals for men. Features stylish webbing straps, Diecut EVA midsole for stable cushioning and abrasion resistant rubber outsole with wet and dry traction pattern. 
These men's running shoes will get you on the road to your goals. A sandwich mesh upper offers lightweight breathability, while a seamless print overlay adds support for a stable stride. The midsole offers pillow-soft Cloudfoam cushioning that eases every stride.
Sleek running shoes for men. These shoes have a lightweight look and feel in honeycomb mesh. Cloudfoam cushioning delivers superior step-in comfort.
The adidas casual slippers for men with a striking camouflage design. Features stylish webbing straps, Diecut EVA midsole for stable cushioning and abrasion resistant rubber outsole with wet and dry traction pattern. 
These men's running shoes keep you comfortable so you sta

2024-08-07 00:43:30,386 - 138031927712640 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


Final Answer:

"Step Up Your Comfort Game"

> Finished chain.
 [2024-08-07 00:43:30][DEBUG]: == [Marketing and Copy writing Head] Task output: "Step Up Your Comfort Game"


 [2024-08-07 00:43:30][DEBUG]: == Working Agent: Senior Marketing and Copy writing Executive
 [2024-08-07 00:43:30][INFO]: == Starting Task: The adidas casual sandals for men. Features stylish webbing straps, Diecut EVA midsole for stable cushioning and abrasion resistant rubber outsole with wet and dry traction pattern. 
These men's running shoes will get you on the road to your goals. A sandwich mesh upper offers lightweight breathability, while a seamless print overlay adds support for a stable stride. The midsole offers pillow-soft Cloudfoam cushioning that eases every stride.
Sleek running shoes for men. These shoes have a lightweight look and feel in honeycomb mesh. Cloudfoam cushioning delivers superior step-in comfort.
The adidas casual slippers for men with a striking camouflage design. Features stylish web

2024-08-07 00:43:32,052 - 138031927712640 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


Thought: I now can give a great answer

Final Answer:

Introducing the adidas casual sandals for men, designed to take your running or jogging sessions to the next level. With its stylish webbing straps, you'll be making a statement on the road. The Diecut EVA midsole provides stable cushioning, while the abrasion-resistant rubber outsole with a wet and dry traction pattern ensures you stay grounded on any surface.

The sandwich mesh upper offers lightweight breathability, allowing your feet to stay cool and dry. The seamless print overlay adds support for a stable stride, giving you the confidence to push yourself to new heights. The midsole features pillow-soft Cloudfoam cushioning that eases every stride, making it feel like you're walking on clouds.

Whether you're a seasoned runner or just starting out, these shoes are designed to keep you comfortable so you can stay focused on your goals. The textile, mesh, and synthetic upper provides a snug fit, while the Cloudfoam midsole prov

In [33]:
print(result)

{'tagline': CrewOutput(raw='"Step Up Your Comfort Game"', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description="The adidas casual sandals for men. Features stylish webbing straps, Diecut EVA midsole for stable cushioning and abrasion resistant rubber outsole with wet and dry traction pattern. \nThese men's running shoes will get you on the road to your goals. A sandwich mesh upper offers lightweight breathability, while a seamless print overlay adds support for a stable stride. The midsole offers pillow-soft Cloudfoam cushioning that eases every stride.\nSleek running shoes for men. These shoes have a lightweight look and feel in honeycomb mesh. Cloudfoam cushioning delivers superior step-in comfort.\nThe adidas casual slippers for men with a striking camouflage design. Features stylish webbing straps, Diecut EVA midsole for stable cushioning and abrasion resistant rubber outsole with wet and dry traction pattern. \nThese men's running shoes keep you comfortable so you s